# Skill estimation using Stan

In [1]:
import numpy as np
import pystan
import matplotlib.pyplot as plt
%matplotlib inline

import pickle

## Model that defined skill's distribution with more features

In [2]:
skill_model = """
data {
  int<lower=1> N;             // Total number of players
  int<lower=1> N1;             // Total number of old players
  int<lower=1> E;             // number of games
  real<lower=0> scale;        // scale value for probability computation
  int<lower=0,upper=1> win[E]; // PA wins vs PB
  int PA[E];                  // player info between each game
  int PB[E];                  // 
  real old_skill[N1];           //
}
parameters {
  vector [N] skill;           // skill values for each player
}

model{
  for (i in 1:N){ 
    if (i <= N1) {skill[i] ~ normal(old_skill[i],1);}
    else {skill[i]~normal(0,3);} 
  }
  for (i in 1:E){
    win[i] ~ bernoulli_logit( (scale)*(skill[PA[i]]-skill[PB[i]]) );
  }   // win probability is a logit function of skill difference
}
"""

Now, compile the model.  

In [3]:
# try:     # load it if already compiled
#     sm = pickle.load(open('skill_model_new.pkl', 'rb'))
# except:  # ow, compile and save compiled model
#     sm = pystan.StanModel(model_code = skill_model)
#     with open('skill_model_new.pkl', 'wb') as f: pickle.dump(sm, f)

In [4]:
sm1 = pickle.load(open('skill_model.pkl', 'rb'))
sm2 = pickle.load(open('skill_model_new.pkl', 'rb'))

## Processing data

In [5]:
def load_data(dir='data/', new=0, pKeep=1.0, nEdge=3, nKeep=5, opt='train'):
    with open(dir+opt+'.csv', encoding='utf-8') as f:
        lines = f.read().split('\n')

    p = 0
    playerid = {}
    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10: 
            continue   # parse error or blank line
        player0,player1 = csv[1],csv[4]
        if player0 not in playerid:
            playerid[player0]=p
            p+=1
        if player1 not in playerid:
            playerid[player1]=p
            p+=1

    
    # Sparsifying parameters (discard some training examples):
    # pKeep = 1.0   # fraction of edges to consider (immed. throw out 1-p edges)
    # nEdge = 3     # try to keep nEdge opponents per player (may be more; asymmetric)
    # nKeep = 5     # keep at most nKeep games per opponent pairs (play each other multiple times)

    wins, wins_n = [], []
    playerA, playerB = [], []
    playerA_n, playerB_n = [], []
    nplayers = len(playerid) - new
    nplayers_n = new 
    nplays = np.zeros( (len(playerid),len(playerid)) )
    
    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10:
            continue   # parse error or blank line
        a,b = playerid[csv[1]],playerid[csv[4]]
        aw,bw = csv[2]=='[winner]',csv[5]=='[winner]'
        if a < nplayers and b < nplayers:
            if (np.random.rand() < pKeep):
                if (nplays[a,b] < nKeep) and ( ((nplays[a,:]>0).sum() < nEdge) or ((nplays[:,b]>0).sum() < nEdge) ):

                    nplays[a,b] += 1
                    nplays[b,a]+=1                

                    playerA.append(a+1)
                    playerB.append(b+1)
                    wins.append(1 if aw else 0) 
        else:
            if (np.random.rand() < pKeep):
                if (nplays[a,b] < nKeep) and ( ((nplays[a,:]>0).sum() < nEdge) or ((nplays[:,b]>0).sum() < nEdge) ):
                    nplays[a,b] += 1
                    nplays[b,a]+=1                      
                    
                    # involve new players
                    playerA_n.append(a+1)
                    playerB_n.append(b+1)
                    wins_n.append(1 if aw else 0) 

    return playerid, nplayers,playerA,playerB,wins,nplayers_n,playerA_n,playerB_n,wins_n

In [6]:
playerid, nplayers,playerA,playerB,wins,nplayers_n,playerA_n,playerB_n,wins_n = load_data(new=1)

In [7]:
print('summary: ')
print('# players', nplayers)
print('# games', len(wins))
print('player A', playerA[:10])
print('player B', playerB[:10])
print('wins', wins[:10])

summary: 
# players 998
# games 4673
player A [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
player B [2, 3, 4, 5, 6, 7, 8, 9, 10, 9]
wins [0, 0, 0, 0, 1, 1, 1, 0, 0, 1]


In [8]:
print('summary: ')
print('# players', nplayers_n)
print('# games', len(wins_n))
print('player A', playerA_n[:10])
print('player B', playerB_n[:10])
print('wins', wins_n[:10])

summary: 
# players 1
# games 4
player A [59, 59, 230, 137]
player B [999, 999, 999, 999]
wins [1, 0, 1, 1]


We also need the observed data: number of players and games, which pairs played each game, and who won:

In [9]:
skill_data = {
    'N': nplayers,
    'E': len(wins),
    'scale': 0.3,
    'win':wins,
    'PA': playerA,
    'PB': playerB
}
# Player 1 & 3 played & P1 won; then again; then P2 & P3 (P2 wins), etc.

In [10]:
fit = sm1.sampling(data=skill_data, iter=1000, chains=4)

In [11]:
samples = fit.extract()

In [12]:
old_skills = samples['skill'].mean(0)
old_skills

array([ 4.48316050e+00,  4.63842087e+00,  6.51987987e+00,  7.88900113e+00,
        6.40907530e+00,  1.59206776e+00,  4.76772418e+00,  1.57181124e+00,
        4.95383529e+00,  6.16241229e+00,  2.13150172e-01,  2.24202080e+00,
        2.49948463e+00, -6.79492840e-01,  4.02472214e+00,  2.68270093e+00,
       -1.26406428e+00, -1.61720798e+00, -5.41870526e-01,  4.82242208e+00,
       -5.47683033e-01,  2.49190592e+00,  3.74292842e+00,  4.85960944e+00,
        5.93069717e+00,  2.08446359e+00,  6.60175180e+00,  2.44356527e-01,
        4.80229583e+00,  2.79198256e+00,  4.64295164e+00, -1.27311634e+00,
        7.93000451e-01,  6.33504460e+00,  4.44014784e-01,  6.39960533e+00,
       -8.77097577e-01,  9.25449144e-01,  5.54048175e+00,  3.91575085e+00,
        8.59247256e-01,  4.86111727e+00,  2.61032583e+00,  4.13131308e+00,
       -9.00103877e-02,  3.94951793e+00,  4.95786787e+00,  5.12937909e+00,
        4.37399444e+00,  2.96005249e+00,  1.70925652e-01,  1.05006832e+00,
        2.29508838e+00, -

new players

In [13]:
skill_data_new = {
    'N': nplayers+nplayers_n,
    'N1': nplayers,
    'E': len(wins_n),
    'scale': 0.3,
    'win':wins_n,
    'PA': playerA_n,
    'PB': playerB_n,
    'old_skill': old_skills
}
# Player 1 & 3 played & P1 won; then again; then P2 & P3 (P2 wins), etc.

Now, we can perform MCMC on the model, and extract the samples:

In [14]:
fit = sm2.sampling(data=skill_data_new, iter=1000, chains=4)

In [15]:
samples = fit.extract()

If we just want the mean estimate for each player's skill level, just take the empirical average over the samples:

In [16]:
samples['skill'].shape # 2*100 iterations? 999 players

(2000, 999)

If we want to predict which player will win, we might use a direct estimator of that quantity based on the sample values:

In [17]:
# Player 0 vs Player 1 prediction:
def logit(z): return 1./(1.+np.exp(-z))

# Use our model's win probability function (logistic of scaled difference)
#  using the predicted skill difference for each sample:
prob = logit( skill_data['scale']*(samples['skill'][:,0]-samples['skill'][:,1]) ).mean()

print(prob)

0.4878126147816312


Remember to save the prediction!

In [18]:
with open('skill_hat_new.pkl', 'wb') as f: 
    pickle.dump(samples['skill'], f)

## Sample Model Evaluation

In [19]:
skill_hat = pickle.load(open('skill_hat_new.pkl', 'rb'))

In [31]:
def load_valid_data(playerid, dir='data/', pKeep=1.0, nEdge=3, nKeep=5, opt='valid'):
    with open(dir+opt+'.csv', encoding='utf-8') as f:
        lines = f.read().split('\n')


  # Sparsifying parameters (discard some training examples):
  # pKeep = 1.0   # fraction of edges to consider (immed. throw out 1-p edges)
  # nEdge = 3     # try to keep nEdge opponents per player (may be more; asymmetric)
  # nKeep = 5     # keep at most nKeep games per opponent pairs (play each other multiple times)
    nplayers = len(playerid)
    games = []
    nplays, nwins = np.zeros( (nplayers,nplayers) ), np.zeros( (nplayers,nplayers) )
    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10:
            continue   # parse error or blank line
        a,b = playerid[csv[1]],playerid[csv[4]]
        aw,bw = csv[2]=='[winner]',csv[5]=='[winner]'
        
        if (np.random.rand() < pKeep):
            if (nplays[a,b] < nKeep) and ( ((nplays[a,:]>0).sum() < nEdge) or ((nplays[:,b]>0).sum() < nEdge) ):
                nplays[a,b] += 1
                nplays[b,a]+=1
                nwins[a,b] += aw
                nwins[b,a] += bw
                games.append((a,b,aw))
    return nplayers, nplays, nwins, games


In [32]:
nplayers_val, nplays_val, nwins_val, games_val = load_valid_data(playerid)

In [33]:
print('summary: ', nplayers_val)
print(nplays_val.shape, nplays_val.sum())
print(nwins_val.shape, nwins_val.sum())
print('games', len(games_val))

summary:  999
(999, 999) 9536.0
(999, 999) 4785.0
games 4768


In [34]:
def logit(z): return 1./(1.+np.exp(-z))

def prediction_loss(skill, nplayers, nplays, nwins, games):
    
    loss = 0.
    binary_loss = 0.
    for i in range(nplayers):
        for j in range(i+1, nplayers):
            if nplays[i, j] == 0:
                continue
            prob = nwins[i,j] / nplays[i,j]
            prob_hat = logit( skill_data['scale']*(skill[:,i]-skill[:,j]) ).mean()
            loss += np.abs(prob_hat - prob)
            binary_loss += np.logical_xor(prob_hat >= 0.5, prob >= 0.5)
    
    loss /= (nplays > 0).sum()/2
    binary_loss /= (nplays > 0).sum()/2
    
    return loss, binary_loss


In [35]:
loss, binary_loss = prediction_loss(skill_hat, nplayers_val, nplays_val, nwins_val, games_val)

In [36]:
loss, binary_loss

(0.2777785979415483, 0.1705270224907687)

In [37]:
def loss_per_game(skill, games):
    
    loss = 0.
    
    for game in games:
        a, b, w = game
        prob_hat = logit( skill_data['scale']*(skill[:,a]-skill[:,b]) ).mean()
        loss += np.logical_xor(prob_hat >= 0.5, w)
        
    loss /= len(games)*2
    
    return loss

In [38]:
loss_per_game(skill_hat, games_val)

0.1055998322147651